In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
def envelope(value):
    # Convert it to a bit array
    bits = []
    for _ in range(64):
        bits.append(value & 1)
        value >>= 1
    bits.reverse()

    # Append the trailer (even parity bit and stop marker)
    parity_bit = 0 if sum(bits) % 2 == 0 else 1
    bits.extend([parity_bit, 0, 1, 1])

    assert(len(bits) == 68)
    assert(sum(bits) % 2 == 0)

    return bits

In [3]:
def create_barcode(bits):
    # Units in 1/100 of a millimeter

    # Create a 28mm by 14mm rectangle with a white background
    img = np.zeros((1400,2800,3), np.uint8)
    img.fill(255)

    # The frame has a thickness of 0.3mm, doubled because we only care about the
    # inner lines
    frame_thickness = 30
    cv2.rectangle(img, (0, 0), (2800, 1400), (0, 0, 0), 2 * frame_thickness)

    # The quiet space, including the frame, is roughly 7mm
    quiet_space_width = 1400 / 2 - frame_thickness
    data_space_begin = (frame_thickness + quiet_space_width, frame_thickness)

    # The data space is roughly 13.6mm-14mm wide. With 68 symbols, that gives us
    # a marker width of 0.2mm-0.205mm.
    marker_width = 20
    marker_height = 1400 - 2 * frame_thickness
    marker_shape = (marker_width, marker_height)

    pos = np.rint(data_space_begin).astype(int)
    for bit in bits:
        # Draw each marker and only fill the inside
        if bit == 1:
            top_left_corner = np.rint(pos).astype(int)
            bottom_right_corner = (top_left_corner + np.rint(marker_shape)).astype(int)
            cv2.rectangle(img, top_left_corner, bottom_right_corner, (0, 0, 0), -1)
        pos[0] += marker_width

    return img

In [4]:
def create_and_save_barcode(value):
    bits = envelope(value)

    img = create_barcode(bits)

    filename = f'barcodes/{value:016x}.png'
    cv2.imwrite(filename, img)

    return filename

In [5]:
# 64-bit value we want to encode (ignoring trailer)
value = 0x93_14_32_F1_73_4C_85_33

outputs = []

# Flip bits:
# for i in range(50, 64):
#     flipped_value = value ^ (1 << i)
#
#     filename = create_and_save_barcode(flipped_value)
#
#     outputs.append((flipped_value, filename))

In [7]:
from fpdf import FPDF
from textwrap import wrap

pdf = FPDF(unit='mm', format='Letter')

pdf.add_page()
pdf.set_font('Arial', size=7)

w = 5
x, y = 0, 0
for value, filename in outputs:
    pdf.cell(10 + 28, 8, ' '.join(wrap(f'{value:016x}', 2)))
    pdf.image(filename, 10 + x * (10 + 28), 10 + 8 + y * (10 + 14), 28, 14)

    x += 1
    if x >= w:
        x = 0
        y += 1

        pdf.ln(10 + 14)

pdf.output('out.pdf', 'F')

''